In [2]:
import os
import numpy as np
import pandas as pd

seed = 10000
np.random.seed(seed)
attack_proportion = [0.2, 0.4, 0.6, 0.8, 1.0]

dataset = "covertype"
loaded_results = np.load(f"/home/zhengyihao/TabularMark/datasets/covertype/watermarked/{dataset}-{seed}.npy", allow_pickle=True).item()
watermarked_data = loaded_results['watermarked_data']
divide_seeds = loaded_results['divide_seeds']
indices = loaded_results['indices']

# 插入函数
def insert_tuples(temp, proportion):
    # 计算需要插入的行数
    num_insertions = int(proportion * len(temp))
    
    # 从原始数据中随机采样新行
    sampled_rows = temp.sample(n=num_insertions, replace=True).reset_index(drop=True)
    
    # 随机生成插入位置
    insertion_indices = np.random.choice(len(temp) + num_insertions, size=num_insertions, replace=False)
    
    # 创建一个新的 DataFrame 用于插入
    expanded_temp = pd.DataFrame(index=range(len(temp) + num_insertions), columns=temp.columns)
    
    # 标记原始数据和新插入数据的位置
    original_indices = np.setdiff1d(np.arange(len(temp) + num_insertions), insertion_indices)
    
    # 插入原始数据和新数据
    expanded_temp.iloc[original_indices] = temp.values
    expanded_temp.iloc[insertion_indices] = sampled_rows.values
    
    return expanded_temp

# 主循环
for proportion in attack_proportion:
    for i in range(1):
        temp = watermarked_data.copy()
        
        # 调用插入函数
        temp = insert_tuples(temp, proportion)
        
        temp.to_csv(f"/home/zhengyihao/TabularMark/datasets/covertype/insertion/{dataset}-{seed}-{proportion}-{i}.csv", index=False)
